In [1]:
import torch

Our system divides the input image into a S × S grid. If the center of an object falls into a grid cell, that grid cell is responsible for detecting that object. 

每张图像分成7x7的grid，每个grid都用于目标检测。

Each grid cell predicts B bounding boxes and confidence scores for those boxes. These confidence scores reflect how confident the model is that the box contains an object and also how accurate it thinks the box is that it predicts.

每个grid预测B个bounding box，每个bounding box类别与置信度
$$confidence = Pr(object)*IOU_{pred}^{truth}$$

https://hackernoon.com/understanding-yolo-f5a74bbc7967
Each grid cell predicts B bounding boxes as well as C class probabilities. The bounding box prediction has 5 components: (x, y, w, h, confidence)  
每个grid cell预测B个包围盒，其中有C类别概率，包围盒有五个部分 (x, y, w, h, confidence)

The (x, y) coordinates represent the center of the box, relative to the grid cell location (remember that, if the center of the box does not fall inside the grid cell, than this cell is not responsible for it). These coordinates are normalized to fall between 0 and 1. The (w, h) box dimensions are also normalized to [0, 1], relative to the image size. Let’s look at an example:  
(x, y)坐标表示了盒子的相对于grid cell的中央，并且，该坐标相当于网格大小归一化，(w, h)相对于图像大小归一化，两个归一化都归一化到[0,1]，如下图所示：  

<p align="center">
  <img width="460" height="300" src="./images/01.png">
</p>

Note that the confidence reflects the presence or absence of an object of any class.  

Now that we understand the 5 components of the box prediction, remember that each grid cell makes B of those predictions, so there are in total S x S x B * 5 outputs related to bounding box predictions.  
一共有$S\times{}S\times{}B\times{}5$个预测值与边界框的预测相关。

It is also necessary to predict the class probabilities, Pr(Class(i) | Object). This probability is conditioned on the grid cell containing one object (see this if you don’t know that conditional probability means). In practice, it means that if no object is present on the grid cell, the loss function will not penalize it for a wrong class prediction, as we will see later. The network only predicts one set of class probabilities per cell, regardless of the number of boxes B. That makes S x S x C class probabilities in total  
这里的Pr(Class(i) | Object)是条件概率。如果一个单元格没有对象存在，损失函数不会因为错误的类预测而惩罚他。不论B值为多少，每个网格只预测一组类概率，最终会生成S x S x C个类概率。

Adding the class predictions to the output vector, we get a S x S x (B * 5 +C) tensor as output.  
最终会有S x S x (B * 5 +C)的张量输出。  
<p align="center">
    <img src="./images/output.png" width="600">
</p>

In [3]:
class MainModel(torch.nn.Module):
    def __init__(self):
        super(MainModel,self).__init__()
        